In [23]:
!pip install xyzservices

In [32]:
import numpy as np
import pandas as pd
import xyzservices.providers as xyz

from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, STAMEN_TERRAIN
from bokeh.models import ColorBar, Select
from bokeh.palettes import RdYlGn
from bokeh.transform import linear_cmap, log_cmap
from bokeh.models.widgets import Slider
from bokeh.layouts import row, column


In [ ]:
fractionOfData = 0.01

In [25]:
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [26]:
train_apartment = pd.read_csv('../data/apartments_train.csv')
train_building = pd.read_csv('../data/buildings_train.csv')
train = pd.merge(train_apartment, train_building, left_on='building_id', right_on='id')
#train.rename(columns={'id_x' : 'apartment_id', 'id_y' : 'building_id'}, inplace=True)
train.rename(columns={'id_x' : 'apartment_id'}, inplace=True)
list = ['id_y', 'seller', 'area_total', 'area_kitchen',
       'area_living', 'floor', 'rooms', 'layout', 'ceiling',
       'bathrooms_shared', 'bathrooms_private', 'windows_court',
       'windows_street', 'balconies', 'loggias', 'condition', 'phones', 'building_id', 'new', 'district', 'street',
       'address', 'constructed', 'material', 'stories', 'elevator_without',
       'elevator_passenger', 'elevator_service', 'parking', 'garbage_chute',
       'heating']
train.drop(list, axis=1, inplace=True)

train.head()

,apartment_id,price,latitude,longitude
0,0,7139520.0,55.544046,37.478055
1,12,4045140.0,55.544046,37.478055
2,55,9500240.0,55.544046,37.478055
3,104,4326660.0,55.544046,37.478055
4,106,4333980.0,55.544046,37.478055


In [33]:
#Establishes color palette and data source.
pal= RdYlGn[10]





source = ColumnDataSource(data=train)

#Establishes color mapper gradient, hover information, and slider widget.
mapper = log_cmap(field_name = "price", palette = pal, low=min(train.price), high=max(train.price))
tooltips = [("Price","@price")]

#Defines scatter plot attributes and add map tile. Implement colors and data points.
fig = figure(x_axis_type = 'mercator', y_axis_type = 'mercator', tooltips = tooltips, title = 'Moscow Residential Housing Prices')
fig.add_tile(get_provider(STAMEN_TERRAIN))
fig.circle(x = 'longitude', y = 'latitude', line_color = mapper,color=mapper, source=source)

#Defines color bar attributes and location
color_bar = ColorBar(color_mapper=mapper['transform'], width=8, location=(0,0))
fig.add_layout(color_bar, 'right')

#Defines layout of graph and widgets
layout = column(fig)

#Calls figure
show(layout)